In [1]:
import numpy as np 
import pandas as pd 
import os, cv2
from tqdm import tqdm
from sklearn.model_selection import StratifiedShuffleSplit

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.applications import *

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
x_train = []
y_train = []

label_map = {   "Black-grass"               :0,
                "Charlock"                  :1,
                "Cleavers"                  :2,
                "Common Chickweed"          :3,
                "Common wheat"              :4,
                "Fat Hen"                   :5,
                "Loose Silky-bent"          :6,
                "Maize"                     :7,
                "Scentless Mayweed"         :8,
                "Shepherds Purse"           :9,
                "Small-flowered Cranesbill" :10,
                "Sugar beet"                :11}

dim = 256

# Preparing training data
dirs = os.listdir("/input/plant_seedlings_classification/train/")
for k in tqdm(range(len(dirs))):    # Directory
    files = os.listdir("/input/plant_seedlings_classification/train/{}".format(dirs[k]))
    for f in range(len(files)):     # Files
        img = cv2.imread('/input/plant_seedlings_classification/train/{}/{}'.format(dirs[k], files[f]))
        targets = np.zeros(12)
        targets[label_map[dirs[k]]] = 1 
        x_train.append(cv2.resize(img, (dim, dim)))
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32)

print(x_train.shape)
print(y_train.shape)

#x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.01, random_state=42)
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.16, random_state=42) # Want a balanced split for all the classes
for train_index, test_index in sss.split(x_train, y_train):
    print("Using {} for training and {} for validation".format(len(train_index), len(test_index)))
    x_train, x_valid = x_train[train_index], x_train[test_index]
    y_train, y_valid = y_train[train_index], y_train[test_index]

100%|██████████| 12/12 [00:55<00:00,  4.88s/it]


(4750, 256, 256, 3)
(4750, 12)


In [4]:
datagen = ImageDataGenerator( horizontal_flip=True, 
                              vertical_flip=True,
                            rescale = 1./255)
                                      
weights = os.path.join('', '/output/weights.h5')

epochs = 100
learning_rate = 0.001
batch_size = 32

callbacks = [ EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
              ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]

base_model = ResNet50(input_shape=(dim, dim, 3), include_top=False, weights='imagenet', pooling='avg') # Average pooling reduces output dimensions
x = base_model.output
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(12, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Train FCN on top with layers frozen
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

94658560/94653016 [==============================] - 14s 0us/step


In [9]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=datagen.flow(x_valid, y_valid, batch_size=batch_size), 
                    validation_steps=len(x_valid)/batch_size,
                    callbacks=callbacks,
                    epochs=21, 
                    verbose=1)

Epoch 1/21
125/124 [==============================] - 68s 545ms/step - loss: 0.4274 - acc: 0.8477 - val_loss: 3.5643 - val_acc: 0.1868
Epoch 2/21
125/124 [==============================] - 69s 548ms/step - loss: 0.4254 - acc: 0.8620 - val_loss: 2.1988 - val_acc: 0.4316
Epoch 3/21
125/124 [==============================] - 69s 548ms/step - loss: 0.4063 - acc: 0.8649 - val_loss: 0.9302 - val_acc: 0.6868
Epoch 4/21
125/124 [==============================] - 68s 547ms/step - loss: 0.4095 - acc: 0.8573 - val_loss: 0.4217 - val_acc: 0.8526
Epoch 5/21
125/124 [==============================] - 68s 547ms/step - loss: 0.4125 - acc: 0.8581 - val_loss: 0.3601 - val_acc: 0.8697
Epoch 6/21
125/124 [==============================] - 69s 548ms/step - loss: 0.4162 - acc: 0.8594 - val_loss: 0.3540 - val_acc: 0.8711
Epoch 7/21
125/124 [==============================] - 68s 548ms/step - loss: 0.4000 - acc: 0.8673 - val_loss: 0.3344 - val_acc: 0.8842
Epoch 8/21
125/124 [==============================] - 6

In [10]:
# fine tune 
# load best weights 

model.load_weights('/output/weights.h5')

for layer in base_model.layers:
    layer.trainable = True

weights = os.path.join('', '/output/finetune_weights.h5')

callbacks = [ EarlyStopping(monitor='val_loss', patience=5, verbose=0), 
              ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]

model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

In [11]:
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=len(x_train)/batch_size, 
                    validation_data=datagen.flow(x_valid, y_valid, batch_size=batch_size), 
                    validation_steps=len(x_valid)/batch_size,
                    callbacks=callbacks,
                    epochs=30, 
                    verbose=1)

Epoch 1/30
125/124 [==============================] - 195s 2s/step - loss: 1.2540 - acc: 0.6630 - val_loss: 4.6617 - val_acc: 0.3697
Epoch 2/30
125/124 [==============================] - 189s 2s/step - loss: 0.4798 - acc: 0.8354 - val_loss: 0.5160 - val_acc: 0.8329
Epoch 3/30
125/124 [==============================] - 187s 1s/step - loss: 0.4232 - acc: 0.8592 - val_loss: 0.7804 - val_acc: 0.7368
Epoch 4/30
125/124 [==============================] - 188s 2s/step - loss: 0.3040 - acc: 0.8964 - val_loss: 0.4777 - val_acc: 0.8579
Epoch 5/30
125/124 [==============================] - 187s 1s/step - loss: 0.2780 - acc: 0.9039 - val_loss: 4.2932 - val_acc: 0.4487
Epoch 6/30
125/124 [==============================] - 187s 1s/step - loss: 0.2738 - acc: 0.9000 - val_loss: 0.5582 - val_acc: 0.8250
Epoch 7/30
125/124 [==============================] - 188s 2s/step - loss: 0.2327 - acc: 0.9202 - val_loss: 2.0754 - val_acc: 0.5855
Epoch 8/30
125/124 [==============================] - 188s 2s/step - 

In [12]:
# ------ TESTING ------
x_test = []
df_test = pd.read_csv('/input/plant_seedlings_classification/sample_submission.csv')

for f, species in tqdm(df_test.values, miniters=100):
    img = cv2.imread('/input/plant_seedlings_classification/test/{}'.format(f))
    x_test.append(cv2.resize(img, (dim, dim)))

x_test = np.array(x_test, np.float32)
print(x_test.shape)

x_test = x_test /255.0

if os.path.isfile(weights):
    model.load_weights(weights)

p_test = model.predict(x_test, verbose = 1)

preds = []
for i in range(len(p_test)):
    pos = np.argmax(p_test[i])
    preds.append(list(label_map.keys())[list(label_map.values()).index(pos)])

df_test['species'] = preds
df_test.to_csv('/output/submission.csv', index=False)

100%|██████████| 794/794 [00:22<00:00, 35.89it/s]


(794, 256, 256, 3)
